# FFNN - Feed Forward Neural Network

Part of **#30DaysOfBasics**, Lets implement a vanilla FFNN with pytorch

In [37]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

from torch.utils.data import Dataset, DataLoader

In [3]:
#load the data

DATA_PATH = '../data/diabetes.csv'

df = pd.read_csv(DATA_PATH)

In [4]:
df.head()

,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure,Triceps skin fold thickness,2-Hour serum insulin,Body mass index,Age,Class
0,6,148,72,35,0,33.6,50,positive
1,1,85,66,29,0,26.6,31,negative
2,8,183,64,0,0,23.3,32,positive
3,1,89,66,23,94,28.1,21,negative
4,0,137,40,35,168,43.1,33,positive


In [22]:
#class mapping to integer

df.class_int = df.Class.apply(lambda x: 1 if x == 'positive'else 0)

/var/folders/fm/r4mdm1_n5s5c4cvws96yn7qm0000gn/T/ipykernel_85216/1751678756.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.class_int = df.Class.apply(lambda x: 1 if x == 'positive'else 0)


In [29]:
#Seperate b/w X, y

X = df.iloc[:,0:-1].values

y = df.class_int.values


print(X.shape, y.shape)

#chang dtype of both

X = X.astype('float32')
y = y.astype('float32')

(768, 7) (768,)


In [56]:
#Make sure Y-label is in 2 dimension

y = torch.from_numpy(y)

y = torch.unsqueeze(y, 1)

print(y.shape)

torch.Size([768, 1])


In [57]:
#feature normalization

norm = StandardScaler()
X = norm.fit_transform(X)

print(X)

[[ 0.6399473   0.8483238   0.14964075 ... -0.6928905   0.20401257
   1.4259955 ]
 [-0.84488505 -1.1233964  -0.16054575 ... -0.6928905  -0.6844219
  -0.1906719 ]
 [ 1.2338802   1.9437239  -0.26394126 ... -0.6928905  -1.1032556
  -0.10558414]
 ...
 [ 0.34298077  0.00330087  0.14964075 ...  0.27959374 -0.73518956
  -0.27575967]
 [-0.84488505  0.1597866  -0.47073224 ... -0.6928905  -0.24020454
   1.1707321 ]
 [-0.84488505 -0.8730192   0.04624526 ... -0.6928905  -0.20212887
  -0.87137395]]


In [58]:
#creating the dataset for our custom data

class CustomDataset(Dataset):
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    
    def __len__(self):
        return len(self.x)

In [59]:
diabetesDataset = CustomDataset(X, y)

In [60]:
#load the dataset into torch's dataloader for batch and shuffling

train_loader = DataLoader(diabetesDataset, batch_size=64, shuffle=True)

print('No of batches: ',len(train_loader))

for feature, out in train_loader:
    print(len(feature), len(out))
    break

No of batches:  12
64 64


In [61]:
#Building the Neural Network

class CustomModel(nn.Module):
    
    def __init__(self, ip_features, op_features):
        super(CustomModel, self).__init__()
        self.ip_features = ip_features
        self.op_features = op_features
        
        self.fc1 = nn.Linear(ip_features, 5)
        self.fc2 = nn.Linear(5, 4)
        self.fc3 = nn.Linear(4, 3)
        self.fc4 = nn.Linear(3,op_features)
        
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
        
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.tanh(out)
        
        out = self.fc2(out)
        out = self.tanh(out)
        
        out = self.fc3(out)
        out = self.tanh(out)
        
        out = self.fc4(out)
        out = self.sigmoid(out)
        
        return out

In [62]:
#declare loss and optimizer function

network = CustomModel(7,1)

#loss
criterion = nn.BCELoss(size_average=True)

#optimizer
optimizer = torch.optim.SGD(network.parameters(), lr=0.01, momentum=0.8)

/Users/impyadav/opt/anaconda3/envs/m1/lib/python3.9/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [64]:
#train the network

epochs = 300

for epoch in range(epochs):
    for feature, label in train_loader:
        
        #just to sure, everything is in float
        feature = feature.float()
        label = label.float()
        
        #forward prop
        outputs = network(feature)
        
        #loss
        loss = criterion(outputs, label)
        
        #clear gradient buffer
        optimizer.zero_grad()
        
        #calcuate gradients
        loss.backward()
        
        #update the weights
        optimizer.step()
        
    #Accuracy calcualtion
    op = (outputs>0.5).float()
    acc = (op == label).float().mean()
    print('{}/{} epoch: Accuracy is {:.3f} and loss is {:.3f}'.format(epoch, epochs, acc, loss))

0/300 epoch: Accuracy is 0.781 and loss is 0.429
1/300 epoch: Accuracy is 0.688 and loss is 0.487
2/300 epoch: Accuracy is 0.750 and loss is 0.509
3/300 epoch: Accuracy is 0.844 and loss is 0.355
4/300 epoch: Accuracy is 0.812 and loss is 0.362
5/300 epoch: Accuracy is 0.797 and loss is 0.377
6/300 epoch: Accuracy is 0.844 and loss is 0.350
7/300 epoch: Accuracy is 0.859 and loss is 0.401
8/300 epoch: Accuracy is 0.750 and loss is 0.493
9/300 epoch: Accuracy is 0.797 and loss is 0.477
10/300 epoch: Accuracy is 0.781 and loss is 0.403
11/300 epoch: Accuracy is 0.750 and loss is 0.446
12/300 epoch: Accuracy is 0.828 and loss is 0.421
13/300 epoch: Accuracy is 0.797 and loss is 0.446
14/300 epoch: Accuracy is 0.781 and loss is 0.373
15/300 epoch: Accuracy is 0.859 and loss is 0.397
16/300 epoch: Accuracy is 0.844 and loss is 0.380
17/300 epoch: Accuracy is 0.891 and loss is 0.317
18/300 epoch: Accuracy is 0.797 and loss is 0.479
19/300 epoch: Accuracy is 0.719 and loss is 0.579
20/300 epo

215/300 epoch: Accuracy is 0.719 and loss is 0.528
216/300 epoch: Accuracy is 0.812 and loss is 0.367
217/300 epoch: Accuracy is 0.750 and loss is 0.437
218/300 epoch: Accuracy is 0.812 and loss is 0.401
219/300 epoch: Accuracy is 0.766 and loss is 0.414
220/300 epoch: Accuracy is 0.750 and loss is 0.426
221/300 epoch: Accuracy is 0.734 and loss is 0.467
222/300 epoch: Accuracy is 0.859 and loss is 0.369
223/300 epoch: Accuracy is 0.812 and loss is 0.431
224/300 epoch: Accuracy is 0.781 and loss is 0.476
225/300 epoch: Accuracy is 0.766 and loss is 0.452
226/300 epoch: Accuracy is 0.781 and loss is 0.487
227/300 epoch: Accuracy is 0.828 and loss is 0.377
228/300 epoch: Accuracy is 0.703 and loss is 0.549
229/300 epoch: Accuracy is 0.812 and loss is 0.393
230/300 epoch: Accuracy is 0.812 and loss is 0.394
231/300 epoch: Accuracy is 0.750 and loss is 0.513
232/300 epoch: Accuracy is 0.766 and loss is 0.442
233/300 epoch: Accuracy is 0.828 and loss is 0.421
234/300 epoch: Accuracy is 0.67